## Fase 3: Diseño de BBDD e Insercción de los Datos

Esta fase tiene como objetivo la creación y la insercción de datos en una base de datos.
- Diseño de la estructura de la Base de Datos Peoplemetrics
- Creación de la Base de Datos
- Insercción de Datos de los empleados de la empresa

In [1]:
#pip install mysql-connector
#pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importamos las librerías que necesitamos
# -----------------------------------------------------------------------

# Librerías para la conexión con MySQL
import mysql.connector
from mysql.connector import errorcode

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

In [2]:
# Función para crear la conexión con MySQL

def crear_conexion (user, password, host):
    
    try:
        cnx = mysql.connector.connect(user=user, password=password, host=host)

    # En caso de que no lo consiga por que hay algún error
    except mysql.connector. Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    
    print("Connection created successfully")   
    return cnx 

# Función que nos permite ejecutar diferentes queries

def execute_query (cnx, query):
    mycursor = cnx.cursor()
    try:
        mycursor.execute(query)
        cnx.commit()
        print("Query executed successfully")
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

# Función para crear la bbdd

def creacion_bbdd (cnx, query):
    execute_query (cnx,query)

# Función para establecer la conexión con la bbdd

def conexion_bbdd (user, password, host, bbdd):
    try:
        cnx_bbdd = mysql.connector.connect(user=user, password=password, host=host, database=bbdd)

    # En caso de que no lo consigas por que hay algún error
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    return cnx_bbdd

In [3]:
# Conexión a MySQL
cnx = crear_conexion ('root', 'AlumnaAdalab', '127.0.0.1')

# Creamos la BBDD en MySQL
query_creacion_bbdd = "CREATE DATABASE IF NOT EXISTS peoplemetrics"
creacion_bbdd(cnx, query_creacion_bbdd)

# Conexión a la BBDD de MySQL
cnx_bbdd = conexion_bbdd ('root', 'AlumnaAdalab', '127.0.0.1', 'peoplemetrics')

Connection created successfully
Query executed successfully


In [4]:
# Query creación de tablas

query_tabla_employees="CREATE TABLE IF NOT EXISTS Employees(Id_employees INT AUTO_INCREMENT PRIMARY KEY,employee_count FLOAT,Gender ENUM('Male', 'Female') NOT NULL,Age INT, Marital_status VARCHAR(25), Over_18 VARCHAR (25), Date_birth INT, Employee_number VARCHAR (25));"

query_tabla_job_details="CREATE TABLE IF NOT EXISTS Job_details(Id_employees INT AUTO_INCREMENT PRIMARY KEY,Job_role VARCHAR(100), Job_level INT,Business_travel VARCHAR(100), Job_involvement INT, Over_time ENUM('Yes', 'No','Unknown') NOT NULL, Remote_work ENUM('Yes', 'No','Unknown') NOT NULL, Years_at_company INT, Years_since_last_promotion INT, Years_with_curr_manager INT);"

query_tabla_employee_profile="CREATE TABLE IF NOT EXISTS Employee_profile (Id_employees INT AUTO_INCREMENT PRIMARY KEY,Education INT,Education_field VARCHAR(100),Num_companies_worked INT,Total_working_years FLOAT);"

query_tabla_salary="CREATE TABLE IF NOT EXISTS Salary (Id_employees INT AUTO_INCREMENT PRIMARY KEY,Hourly_rate FLOAT, Daily_rate FLOAT, Monthly_rate FLOAT, Percent_salary_hike FLOAT, Stock_option_level INT,Training_times_last_year INT);"

query_tabla_employee_company="CREATE TABLE IF NOT EXISTS Employee_company(Id_employees INT AUTO_INCREMENT PRIMARY KEY,Attrition ENUM('Yes', 'No') NOT NULL,Environment_satisfaction INT,Job_satisfaction INT,Performance_rating FLOAT,Relationship_satisfaction INT,Work_life_balance FLOAT,Distance_from_home FLOAT);"

In [5]:
# Creamos las tablas

execute_query(cnx_bbdd,query_tabla_employees)
execute_query(cnx_bbdd,query_tabla_job_details)
execute_query(cnx_bbdd,query_tabla_employee_profile)
execute_query(cnx_bbdd,query_tabla_salary)
execute_query(cnx_bbdd,query_tabla_employee_company)

print("Tables successfully created")

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Tables successfully created


In [6]:
# Creamos la función 'cargar datos' para insertar los datos en las tablas de la base de datos

def cargar_datos (user, password, host, bbdd, query, lista_tuplas):
    cnx = mysql.connector.connect(user=user, password=password, host=host, database=bbdd)
    mycursor = cnx.cursor()
    try:
          mycursor = cnx.cursor()
          mycursor.executemany(query, lista_tuplas)
          cnx.commit()
          print(mycursor.rowcount, "row/s inserted successfully")
          cnx.close()
    except mysql.connector.Error as err:
          print(err)
          print("Error Code:", err.errno)
          print("SQLSTATE", err.sqlstate)
          print("Message", err.msg)
          cnx.close()


url = "https://raw.githubusercontent.com/s-armeni/proyecto-da-promo-H-modulo-3-team-3-DataMinds/main/hr_data_final.csv"

# Cargamos el csv

df=pd.read_csv(url, index_col=0)

# Generamos listas de tuplas con la informarción a insertar para poder insertar los datos

datos_tabla_employees=list(zip(df["Employee_count"].values, df["Gender"].values, df["Age"].values, df["Marital_status"].values, df["Over_18"].values,df["Date_birth"].values, df["Employee_number"].values))
datos_tabla_employee_profile=list(zip(df["Education"].values, df["Education_field"].values,df["Num_companies_worked"].values,df["Total_working_years"].values))
datos_tabla_job_details=list(zip(df["Job_role"].values, df["Job_level"].values, df["Business_travel"].values, df["Job_involvement"].values, df["Over_time"].values,df["Remote_work"].values, df["Years_at_company"].values,df["Years_since_last_promotion"].values, df["Years_with_curr_manager"]))
datos_tabla_salary=list(zip(df["Hourly_rate"].values, df["Daily_rate"].values, df["Monthly_rate"].values, df["Percent_salary_hike"].values, df["Stock_option_level"].values,df["Training_times_last_year"].values))
datos_tabla_employee_company=list(zip(df["Attrition"].values, df["Environment_satisfaction"].values, df["Job_satisfaction"].values, df["Performance_rating"].values, df["Relationship_satisfaction"].values,df["Work_life_balance"].values,df["Distance_from_home"].values))

# Query insercción datos en tablas

query_cargar_datos_employees="INSERT INTO Employees (Employee_count, Gender, Age, Marital_status, Over_18, Date_birth, Employee_number) VALUES (%s, %s, %s, %s, %s, %s, %s)"
query_cargar_datos_employee_profile="INSERT INTO Employee_profile (Education, Education_field, Num_companies_worked, Total_working_years) VALUES (%s, %s, %s, %s)"
query_cargar_datos_job_details="INSERT INTO Job_details (Job_role,Job_level,Business_travel,Job_involvement,Over_time,Remote_work,Years_at_company,Years_since_last_promotion,Years_with_curr_manager) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
query_cargar_datos_salary="INSERT INTO Salary (Hourly_rate,Daily_rate,Monthly_rate,Percent_salary_hike,Stock_option_level,Training_times_last_year) VALUES (%s,%s,%s,%s,%s,%s)"
query_cargar_datos_employee_company="INSERT INTO Employee_company(Attrition,Environment_satisfaction,Job_satisfaction,Performance_rating,Relationship_satisfaction,Work_life_balance,Distance_from_home) VALUES (%s,%s,%s,%s,%s,%s,%s)"

In [7]:
# Función para convertir los elementos de una lista de tuplas a int 

def convertir_int(lista_tuplas):
    datos_tabla_caract_def = []
    for tupla in lista_tuplas:
        lista_intermedia = []
        for elemento in tupla:
            try:
                lista_intermedia.append(int(elemento))
            except:
                lista_intermedia.append(elemento)
            
        datos_tabla_caract_def.append(tuple(lista_intermedia))
    
    return datos_tabla_caract_def

In [8]:
# Llamamos la función para convertir los elementos de las tuplas

datos_employee_profile= convertir_int(datos_tabla_employee_profile)
datos_employees=convertir_int(datos_tabla_employees)
datos_job_details=convertir_int(datos_tabla_job_details)
datos_salary=convertir_int(datos_tabla_salary)

In [9]:
# Función para convertir los elementos de tipo numpy.float a float

def convertir_float(lista_tuplas):
    datos_tabla_caract_def = []
    for tupla in lista_tuplas:
        lista_intermedia = []
        for elemento in tupla:
            try:
                lista_intermedia.append(float(elemento))
            except:
                lista_intermedia.append(elemento)
            
        datos_tabla_caract_def.append(tuple(lista_intermedia))
    
    return datos_tabla_caract_def

In [10]:
# Llamamos la función para convertir los elementos de las tuplas
datos_employee_company=convertir_float(datos_tabla_employee_company)

In [11]:
# Llamamos a la función cargar_datos para insertar los datos en las tablas 

In [12]:
cargar_datos('root', 'AlumnaAdalab', '127.0.0.1', 'peoplemetrics',query_cargar_datos_employee_profile,datos_employee_profile)

1614 row/s inserted successfully


In [13]:
cargar_datos('root', 'AlumnaAdalab', '127.0.0.1', 'peoplemetrics',query_cargar_datos_employees,datos_employees)

1614 row/s inserted successfully


In [14]:
cargar_datos('root', 'AlumnaAdalab', '127.0.0.1', 'peoplemetrics', query_cargar_datos_job_details,datos_job_details) 

1614 row/s inserted successfully


In [15]:
cargar_datos('root', 'AlumnaAdalab', '127.0.0.1', 'peoplemetrics', query_cargar_datos_salary, datos_salary)

1614 row/s inserted successfully


In [16]:
cargar_datos('root', 'AlumnaAdalab', '127.0.0.1', 'peoplemetrics', query_cargar_datos_employee_company, datos_employee_company)

1614 row/s inserted successfully
